Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [121]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [122]:
import pandas as pd
import pandas_profiling
import numpy as np

# Read New York City property sales data
df = pd.read_csv(DATA_PATH +'condos/NYC_Citywide_Rolling_Calendar_Sales.csv', parse_dates= ['SALE DATE'], index_col= 'SALE DATE')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [123]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [124]:
import numpy as np
# As per instruction, we also need to take a look at the BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS'

df = df[df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS']
# WE are goig to take a look at what all this shenanigans has produced for us


# Reduce cardinality for ALL features

high_cardinality = [col for col in df.select_dtypes('object').columns 
                      if df[col].nunique() > 10]
df.drop(columns=high_cardinality, inplace=True)

  # Remove use prices between 100,000 and 2,000,000 dollars
df = df[(df['SALE_PRICE'] > 10000) &  (df['SALE_PRICE'] <= 2000000)]

In [125]:

# WE are goig to take a look at what all this shenanigans has produced for us

df



,BOROUGH,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE
SALE DATE,,,,,,,,,,,,,,,
2019-01-01,3,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,NaN,11230.0,1.0,0.0,1.0,1325.0,1930.0,1,550000
2019-01-01,4,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,NaN,11427.0,1.0,0.0,1.0,2001.0,1940.0,1,200000
2019-01-02,2,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,NaN,10461.0,1.0,0.0,1.0,2043.0,1925.0,1,810000
2019-01-02,3,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,NaN,11226.0,1.0,0.0,1.0,2680.0,1899.0,1,125000
2019-01-02,3,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,NaN,11203.0,1.0,0.0,1.0,1872.0,1940.0,1,620000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-04-30,4,01 ONE FAMILY DWELLINGS,1,13215,3,NaN,NaN,11422.0,1.0,0.0,1.0,1478.0,1925.0,1,635000
2019-04-30,4,01 ONE FAMILY DWELLINGS,1,11612,73,NaN,NaN,11420.0,1.0,0.0,1.0,1280.0,1930.0,1,514000
2019-04-30,4,01 ONE FAMILY DWELLINGS,1,11808,50,NaN,NaN,11420.0,1.0,0.0,1.0,1333.0,1945.0,1,635000


In [126]:
print(df.shape)
print(df.info())

(3196, 15)
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3196 entries, 2019-01-01 to 2019-04-30
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   BOROUGH                    3196 non-null   object 
 1   BUILDING_CLASS_CATEGORY    3196 non-null   object 
 2   TAX_CLASS_AT_PRESENT       3196 non-null   object 
 3   BLOCK                      3196 non-null   int64  
 4   LOT                        3196 non-null   int64  
 5   EASE-MENT                  0 non-null      float64
 6   APARTMENT_NUMBER           1 non-null      object 
 7   ZIP_CODE                   3196 non-null   float64
 8   RESIDENTIAL_UNITS          3196 non-null   float64
 9   COMMERCIAL_UNITS           3196 non-null   float64
 10  TOTAL_UNITS                3196 non-null   float64
 11  GROSS_SQUARE_FEET          3196 non-null   float64
 12  YEAR_BUILT                 3196 non-null   float64
 13  TAX_CLASS_AT_TIME_O

In [127]:
# will drop the 'Ease-Ment' column just a bunch of nulls and APARTMENT_NUMBER

print(df.columns)

df.drop('EASE-MENT', axis = 1, inplace = True)
df.drop('APARTMENT_NUMBER', axis = 1, inplace = True)


Index(['BOROUGH', 'BUILDING_CLASS_CATEGORY', 'TAX_CLASS_AT_PRESENT', 'BLOCK',
       'LOT', 'EASE-MENT', 'APARTMENT_NUMBER', 'ZIP_CODE', 'RESIDENTIAL_UNITS',
       'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'SALE_PRICE'],
      dtype='object')


In [128]:
df

,BOROUGH,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE
SALE DATE,,,,,,,,,,,,,
2019-01-01,3,01 ONE FAMILY DWELLINGS,1,5495,801,11230.0,1.0,0.0,1.0,1325.0,1930.0,1,550000
2019-01-01,4,01 ONE FAMILY DWELLINGS,1,7918,72,11427.0,1.0,0.0,1.0,2001.0,1940.0,1,200000
2019-01-02,2,01 ONE FAMILY DWELLINGS,1,4210,19,10461.0,1.0,0.0,1.0,2043.0,1925.0,1,810000
2019-01-02,3,01 ONE FAMILY DWELLINGS,1,5212,69,11226.0,1.0,0.0,1.0,2680.0,1899.0,1,125000
2019-01-02,3,01 ONE FAMILY DWELLINGS,1,7930,121,11203.0,1.0,0.0,1.0,1872.0,1940.0,1,620000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-04-30,4,01 ONE FAMILY DWELLINGS,1,13215,3,11422.0,1.0,0.0,1.0,1478.0,1925.0,1,635000
2019-04-30,4,01 ONE FAMILY DWELLINGS,1,11612,73,11420.0,1.0,0.0,1.0,1280.0,1930.0,1,514000
2019-04-30,4,01 ONE FAMILY DWELLINGS,1,11808,50,11420.0,1.0,0.0,1.0,1333.0,1945.0,1,635000


# II. Split Data

Split **target vector** from **feature matrix**.

In [129]:
target = 'SALE_PRICE'

y = df[target]
X = df.drop(columns = target)

In [130]:
y


SALE DATE
2019-01-01    550000
2019-01-01    200000
2019-01-02    810000
2019-01-02    125000
2019-01-02    620000
               ...  
2019-04-30    635000
2019-04-30    514000
2019-04-30    635000
2019-04-30    545000
2019-04-30    510000
Name: SALE_PRICE, Length: 3196, dtype: int64

In [131]:
cutoff = '2019-04-01'
mask = X.index < cutoff

X_train, y_train = X.loc[mask], y.loc[mask]
X_test, y_test = X.loc[~mask], y.loc[~mask]


# III. Establish Baseline


In [132]:
from sklearn.metrics import mean_absolute_error
y_pred = [y_train.mean()] * len(y_train)
print('Mean price', y_train.mean())
print('Baseline MAE', mean_absolute_error(y_train, y_pred))

Mean price 614722.3406679764
Baseline MAE 219051.4830821249


# IV. BUILD MODELS

In [133]:
from sklearn.linear_model import LinearRegression, Ridge
from category_encoders import OneHotEncoder
from sklearn.feature_selection import SelectKBest


In [134]:
#Instantiate the existence of the models in the notebook

ohe = OneHotEncoder(use_cat_names= True)
skb = SelectKBest(k = 8 )

In [135]:
#Fit the transformer to the Training data, to transform X_train determining which variables are categorical
ohe.fit(X_train)

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


OneHotEncoder(cols=['BOROUGH', 'BUILDING_CLASS_CATEGORY',
                    'TAX_CLASS_AT_PRESENT'],
              drop_invariant=False, handle_missing='value',
              handle_unknown='value', return_df=True, use_cat_names=True,
              verbose=0)

In [138]:
#Transform the training data, taking those categorical variables and splitting them into new features

XT_train = ohe.transform(X_train)
XT_test = ohe.transform(X_test)

In [137]:
#Transform also using SelectK Best, to find the best features
skb.fit(XT_train, y_train)
XTT_train = skb.transform(XT_train)
XTT_test = skb.transform(XT_test)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 4  5 16] are constant.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [139]:
#Instantiate the Linear Regression model
model_lr = LinearRegression()
#Fit the linear model to the training data
model_lr.fit(XTT_train, y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)